# License Plate Read

In [1]:
import sys
sys.path.append('..')


In [2]:
import license as lc
import cv2
import numpy as np
import random

## Data

Unzip the data (it takes few time)

In [3]:
source_path = '../data/characterData.zip'
target_path = './data/license'

In [4]:
lc.unzip_data(source_path, target_path)

文件已解压


In [5]:
classdict = lc.get_classdict(target_path)

Get data (it takes few time)

In [6]:
data=[]
for classn in classdict.keys():
    data += lc.get_class_data(classdict, classn)
    print('class:' + classn + ', size:' + str(len(data)))

class:0, size:715
class:1, size:1232
class:2, size:1833
class:3, size:2420
class:4, size:2821
class:5, size:3794
class:6, size:4552
class:7, size:5376
class:8, size:6528
class:9, size:7694
class:A, size:8707
class:B, size:9129
class:C, size:9429
class:cuan, size:9555
class:D, size:9768
class:E, size:10067
class:e1, size:10130
class:F, size:10368
class:G, size:10634
class:gan, size:10714
class:gan1, size:10760
class:gui, size:10813
class:gui1, size:10845
class:H, size:11058
class:hei, size:11159
class:hu, size:11362
class:J, size:11526
class:ji, size:11621
class:jin, size:11861
class:jing, size:12008
class:jl, size:12060
class:K, size:12326
class:L, size:12591
class:liao, size:12670
class:lu, size:12774
class:M, size:13017
class:meng, size:13070
class:min, size:13137
class:N, size:13340
class:ning, size:13372
class:P, size:13550
class:Q, size:13676
class:qing, size:13712
class:qiong, size:13742
class:R, size:13907
class:S, size:14071
class:shan, size:14144
class:su, size:14363
class:sx,

## Model

In [7]:
import sys
sys.path.append("..")
import ecust_autodiff as ad

In [8]:
x = ad.Variable(name = "x")
W = ad.Variable(name = "W")
b = ad.Variable(name = "b")
y_ = ad.Variable(name = "y_")

y = ad.matmul_op(x,W) + b

loss = ad.softmax_xentropy_op(y, y_)

grad_W, grad_b = ad.gradients(loss, [W, b])

executor = ad.Executor([loss, grad_W, grad_b])

In [9]:
classn = len(classdict)

Ws = np.zeros([400, classn])
bs = np.zeros([classn])

lr_W = 1e-4
lr_b = 1e-4

for i in range(30000):
    input_data = random.sample(data, 100)
    input_x = []
    input_y = []
    for j in range(100):
        input_x.append(input_data[j][0])
        input_y.append(input_data[j][1])
    
    input_xs = np.array(input_x)
    input_ys = np.array(input_y)

    loss_val, grad_W_val, grad_b_val = executor.run(feed_dict = {
        x: input_xs,
        W: Ws,
        b: bs,
        y_: input_ys
    })
    
    Ws -= grad_W_val * lr_W
    bs -= np.mean(grad_b_val, 0) * lr_b

    if i%100==0:
        print('n=' + str(i) + ', loss=' + str(loss_val))

n=0, loss=[4.17438727]
n=100, loss=[3.17849946]
n=200, loss=[2.72896775]
n=300, loss=[2.52794853]
n=400, loss=[2.2710435]
n=500, loss=[2.2431232]
n=600, loss=[1.90903992]
n=700, loss=[1.79649247]
n=800, loss=[1.75198684]
n=900, loss=[1.7413499]
n=1000, loss=[1.34914912]
n=1100, loss=[1.4156309]
n=1200, loss=[1.54522235]
n=1300, loss=[1.29530986]
n=1400, loss=[1.26826574]
n=1500, loss=[1.16055271]
n=1600, loss=[1.29625816]
n=1700, loss=[0.98072399]
n=1800, loss=[1.09383867]
n=1900, loss=[1.27615478]
n=2000, loss=[0.95488926]
n=2100, loss=[1.12621575]
n=2200, loss=[1.02748699]
n=2300, loss=[1.13898588]
n=2400, loss=[0.82217308]
n=2500, loss=[0.91114819]
n=2600, loss=[0.9421236]
n=2700, loss=[0.95736002]
n=2800, loss=[0.79028872]
n=2900, loss=[0.86138424]
n=3000, loss=[0.75509598]
n=3100, loss=[0.66051644]
n=3200, loss=[0.73876155]
n=3300, loss=[0.81768348]
n=3400, loss=[1.05663297]
n=3500, loss=[0.62358347]
n=3600, loss=[0.67452379]
n=3700, loss=[0.78535906]
n=3800, loss=[0.68188582]
n=3

## Test


In [10]:
timg = lc.get_image('../data/test/0.png')

In [11]:
test = ad.Executor([y])

In [12]:
ps = test.run(feed_dict = {
    x: timg,
    W: Ws,
    b: bs
})
list(classdict.keys())[np.argmax(ps)]

'su'